# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
query = """
CREATE KEYSPACE IF NOT EXISTS user_activity
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
"""

try:
    session.execute(query)
except Exception as e:
    print(e)
else:
    print("Success to create keyspace.")

Success to create keyspace.


#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('user_activity')
except Exception as e:
    print(e)
else:
    print("Success to set keyspace.")

Success to set keyspace.


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## DML functions

- drop_table: dropping table.
- create_table: creating table.
- insert_into_table: inserting user activity into table.
- select_from_table: select data from table.

In [8]:
def drop_table(drop_query):
    """Drop table.
    
    Args:
        drop_query str, query to drop table
    """
    try:
        session.execute(drop_query)
    except Exception as e:
        print(e)
    else:
        print("Success to drop table.")

In [9]:
def create_table(create_query, drop_query=None):
    """Create table. If dropping table is needed, drop table beforehand.

    Args:
        create_query: str, query to create table.
        drop_query: str, query to drop table (default None).
    """
    # Execute drop_table_query if drop_table_query is not None.
    if drop_query is not None:
        drop_table(drop_query)

    # Execute create_table_query
    try:
        session.execute(create_query)
    except Exception as e:
        print(e)
    else:
        print("Success to create table.")

In [10]:
def insert_into_table(insert_query, index_list, type_list):
    """Insert data into table. A number of inserting columns must be
    the same as the length of index_list.
    
    Args:
        insert_query: str, query to insert data into table.
        index_list: list, list of indexes of data inserted into table.
        type_list: list, list of types of inserted data.
    """
    # We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
            ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            try:
                session.execute(insert_query, (type_list[i](line[idx]) for i, idx in enumerate(index_list)))
            except Exception as e:
                print(e)
        
        print("Success to insert into table.")

In [11]:
def select_from_table(select_query, print_format):
    """Select data from table. A number of results of select query must be
    the same as the length of fields indicated by print_format.
    
    Args:
        select_query: str, query to select data from table.
        print_format: str, print format for results.
    """
    try:
        rows = session.execute(select_query)
        for r in rows:
            print (print_format.format(*list(r)))
    except Exception as e:
        print(e)
    else:
        print("Success to select for table.")

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To select specific music app history by using session_id, Primary key (partition key) is set to session_id, and clustering column is set to user_id and item_in_session because data must become uniqe by user and item in session.

- TABLE
    - session_id int
    - user_id
    - item_in_session int
    - artist_text
    - song text
    - length float
- PRIMARY KEY
    - session_id
- CLUSTERING CLUMN
    - user_id
    - item_in_session

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To select spesific user activity by using user_id and session_id, Primary key (partition key) is set to (user_id, session_id). Clustering columns must be set to item_in_session because data is sorted by item_in_session.

- TABLE
    - user_id int
    - session_id int
    - item_in_session int
    - artist text
    - song text
    - first_name text
    - last_name text 
- PRIMARY KEY
    - user_id
    - session_id
- CLUSTERING COLUMN
    - item_in_session

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To select spesific user by using song, Primary key (partition key) is set to song. Clustering column is user_id because data must become uniqe by user.

- TABLE 
    - song text
    - user_id int
    - first_name text
    - last_name text
- PRIMARY KEY
    - song
- CLUSTERING COLUMN
    - user_id

In [12]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

##############################
# Create table.
##############################
drop_query = "DROP TABLE IF EXISTS music_app_history"

create_query = """
CREATE TABLE IF NOT EXISTS music_app_history (
    session_id int
    ,user_id int
    ,item_in_session int
    ,artist text
    ,song text
    ,length float
    ,PRIMARY KEY (session_id, item_in_session)
)
"""

create_table(create_query, drop_query)

##############################
# Insert into table.
##############################
insert_query = """
INSERT INTO music_app_history (
    session_id
    ,user_id
    ,item_in_session
    ,artist
    ,song
    ,length
)
VALUES (%s, %s, %s, %s, %s, %s)
"""

index_list = [8, 10, 3, 0, 9, 5]
type_list = [int, int, int, str, str, float]

insert_into_table(insert_query, index_list, type_list)

Success to drop table.
Success to create table.
Success to insert into table.


#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query = """
SELECT
    artist
    ,song
    ,length
FROM
    music_app_history
WHERE
    session_id=338
    AND item_in_session=4
"""

print_format = "the artist: {}, song title: {}, song's length: {}."

select_from_table(select_query, print_format)

the artist: Faithless, song title: Music Matters (Mark Knight Dub), song's length: 495.30731201171875.
Success to select for table.


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

##############################
# Create table.
##############################
drop_query = "DROP TABLE IF EXISTS user_activity"

create_query = """
CREATE TABLE IF NOT EXISTS user_activity (
    user_id int
    ,session_id int
    ,item_in_session int
    ,artist text
    ,song text
    ,first_name text
    ,last_name text
    ,PRIMARY KEY ((user_id, session_id), item_in_session)
)
WITH CLUSTERING ORDER BY (item_in_session DESC)
"""

create_table(create_query, drop_query)

##############################
# Insert into table.
##############################
insert_query = """
INSERT INTO user_activity (
    user_id
    ,session_id
    ,item_in_session
    ,artist
    ,song
    ,first_name
    ,last_name
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

index_list = [10, 8, 3, 0, 9, 1, 4]
type_list = [int, int, int, str, str, str, str]

insert_into_table(insert_query, index_list, type_list)   

Success to drop table.
Success to create table.
Success to insert into table.


In [15]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query = """
SELECT
    artist
    ,song
    ,first_name
    ,last_name
FROM
    user_activity
WHERE
    user_id=10
    AND session_id=182
"""

print_format = "the artist: {}, song title: {}, first name: {}, last name: {}."

select_from_table(select_query, print_format)

the artist: Lonnie Gordon, song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), first name: Sylvie, last name: Cruz.
the artist: Sebastien Tellier, song title: Kilometer, first name: Sylvie, last name: Cruz.
the artist: Three Drives, song title: Greece 2000, first name: Sylvie, last name: Cruz.
the artist: Down To The Bone, song title: Keep On Keepin' On, first name: Sylvie, last name: Cruz.
Success to select for table.


In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##############################
# Create table.
##############################
drop_query = "DROP TABLE IF EXISTS user_listening_song"

create_query = """
CREATE TABLE IF NOT EXISTS user_listening_song (
    song text
    ,user_id int
    ,first_name text
    ,last_name text
    ,PRIMARY KEY (song, user_id)
)
"""

create_table(create_query, drop_query)

##############################
# Insert into table.
##############################
insert_query = """
INSERT INTO user_listening_song (
   song
   ,user_id
   ,first_name
   ,last_name
)
VALUES (%s, %s, %s, %s)
"""

index_list = [9, 10, 1, 4]
type_list = [str, int, str, str]

insert_into_table(insert_query, index_list, type_list)

Success to drop table.
Success to create table.
Success to insert into table.


In [17]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query = """
SELECT
    first_name
    ,last_name
FROM
    user_listening_song
WHERE
    song='All Hands Against His Own'
"""

print_format = "first name: {}, last name: {}."

select_from_table(select_query, print_format)

first name: Jacqueline, last name: Lynch.
first name: Tegan, last name: Levine.
first name: Sara, last name: Johnson.
Success to select for table.


### Drop the tables before closing out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions
drop_table("DROP TABLE IF EXISTS music_app_history")
drop_table("DROP TABLE IF EXISTS user_activity")
drop_table("DROP TABLE IF EXISTS user_listening_song")

Success to drop table.
Success to drop table.
Success to drop table.


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()